In [25]:
import pandas as pd
import numpy as np
from random import shuffle
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
import os
import scipy
import sqlite3
from collections import defaultdict
import re


def find_results(tp,y_t):
    
    results = list(zip(tp,y_t))
    correct = 0

    for i in range(len(results)):
        if results[i][0] == results[i][1]:
            correct +=1
    
    return(correct/float(len(results)))



def home_own_look(home_val):
    re_v = 0
    if home_val == 'OWN':
        re_v = 1
    return re_v

In [2]:
curr_path = os.path.realpath('')
print(1)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
print(2)
# very important, select a random sample of the data for our training set & validation sets 
# 
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 800000);", conn)
print(3)
df_default = pd.read_sql_query( "select * from loan where loan_status='Charged Off' OR loan_status='Default' OR loan_status='Late (31-120 days)' OR loan_status='Late (16-30 days)' OR loan_status='Does not meet the credit policy. Status:Charged Off' AND id in(select id from loan order by random());", conn)
print(4)
df_fully_paid = pd.read_sql_query("select * from loan where loan_status='Issued' OR loan_status='Fully Paid' OR loan_status='In Grace Period' OR loan_status='Does not meet the credit policy. Status:Fully Paid' AND id in(select id from loan order by random());" , conn)
print(5)
# df_default = pd.read_sql_query("select",conn)
conn.close()
# df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
# df_validation = pd.read_sql_query("select * from loan where id in(selec id from loan order by random() limit 100000);", conn)
# print(df)

1
2
3
4
5


In [3]:
# grab a 50,000 test set
# df_test = df.sample(n = 50000).reset_index(drop=True)
# df_test.to_csv('X_test_set.csv', encoding='utf-8')
# write the 50,000 test set to a csv file
df_test = pd.read_csv('X_test_set.csv', encoding='utf-8')
df_test.fillna(value = np.nan, inplace=True)

In [4]:
# grab a random set of 25,000 "default" examples
df_default_pick = df_default.sample(n = 50000).reset_index(drop=True)
# grab a random set of 25,000 "fully paid" examples
df_fully_paid_pick = df_fully_paid.sample(n = 50000).reset_index(drop=True)

df_for_validation = df.sample(n = 100000).reset_index(drop=True)
df_pick = df_default_pick.append(df_fully_paid_pick)
#df_def_1 = df_default_pick.iloc[:int(len(df_default_pick)/2), :]
#df_def_2 = df_default_pick.iloc[int(len(df_default_pick)/2):, :]
#df_fp_1 = df_fully_paid_pick.iloc[:int(len(df_fully_paid_pick)/2), :]
#df_fp_2 = df_fully_paid_pick.iloc[int(len(df_fully_paid_pick)/2):, :]
# append the training sets together first 25,000 defaults, next 25,000 non-defaults
# dt1 = df_def_1.append(df_fp_1)
# dv1 = df_def_2.append(df_fp_2)
# convert the NaNs to None
# data_train = dt1.where((pd.notnull(dt1)), None)
# data_validation = dv1.where((pd.notnull(dv1)), None)
data_train = df_pick
data_validation = df_for_validation
data_train.fillna(value = np.nan, inplace=True)
data_validation.fillna(value = np.nan, inplace=True)

# print(len(df_default))
# print(len(df_fully_paid))
# print(len(data_train))
print(data_train)


        index        id   member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0      151674  59591750  63525497.0    20000.0      20000.0     20000.000000   
1       13959    773791    959602.0    32000.0      32000.0     32000.000000   
2      145771   1682051   1964694.0    21825.0      21825.0     21825.000000   
3      125984   3238917   3981619.0     8000.0       8000.0      8000.000000   
4      114383   3696098   4668475.0     5575.0       5575.0      5525.000000   
5       28052  34372484  37035787.0     3600.0       3600.0      3600.000000   
6       10650    830027   1039166.0    15775.0      15775.0     15750.000000   
7       40482    509010    656884.0     4200.0       4200.0      4200.000000   
8        1156  37661000  40423893.0    10500.0      10500.0     10450.000000   
9       94715   5041314   6343817.0    29000.0      29000.0     28975.000000   
10     104134   4291200   5483908.0    20000.0      20000.0     20000.000000   
11     187667   1077977   1314324.0    2

In [32]:
# set up 

sub_g_labels = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3'\
            ,'D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']

# make sub_grade 
# give sub grade numeric value.
grade_v = [(1.0/35 * i) for i in range(1, 36)]

# dictionary to make sub_grade val to sub_grade string
# sub_grade_dict = dict(zip(sub_g_labels, grade_v))
sub_grade_dict = dict(zip(sub_g_labels, range(1, 36)))
sub_grade_dict[np.nan] = np.nan

# print(sub_grade_dict)
# print(df.axes)

In [49]:
ax = ['Index','index','id','member_id','loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade'\
      ,'emp_title','emp_length','home_ownership','annual_inc','verification_status','issue_d','loan_status','pymnt_plan','url','desc','purpose','title','zip_code','addr_state','dti'\
      ,'delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec'\
      ,'revol_bal','revol_util','total_acc','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv'\
      ,'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'\
      ,'next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','mths_since_last_major_derog','policy_code','application_type'\
      ,'annual_inc_joint','dti_joint','verification_status_joint','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_il_6m'\
      ,'open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc'\
      ,'all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m']

look = dict(zip(ax, range(len(ax))))
ls = 'loan_status'



def feature(datum):
    # create our simplistic featur vector
    sg = sub_grade_dict[datum[look['sub_grade']]]
    la = float(datum[look['loan_amnt']])
    dti = float(datum[look['dti']])
    ho = home_own_look(datum[look['home_ownership']])
    ai = float(datum[look['annual_inc']])
    # ir = datum[look['int_rate']].split('%')[0]
    #if datum[look['int_rate']] == np.nan:
        # do nothing
        #ir = np.nan
    #else:
        #ir = datum[look['int_rate']].split('%')[0]
    # print(type(ir))
    # ir = float(datum[look['int_rate']])
    return [sg,la,dti,ho,ai]
    # return [sub_grade_dict[datum[look['sub_grade']]], datum[look['loan_amnt']]]


    

def prediction_of_y(datum):
    if ((datum[look[ls]] == 'Charged Off') or (datum[look[ls]] == 'Default') or (datum[look[ls]] == 'Late (31-120 days)' ) or (datum[look[ls]] == 'Late (16-30 days)') or (datum[look[ls]] =='Does not meet the credit policy. Status:Charged Off')):
        return True
    else:
        return False

# X_train = [feature(d) for d in data_train]
X_train = []
pp = True
for row in data_train.itertuples():
    if pp: 
        # print(row)
        pp = False
    X_train.append(feature(row))
    
# y_train = [prediction_of_y(d) for d in data_train]
y_train = []
for row in data_train.itertuples():
    y_train.append(prediction_of_y(row))
    

In [50]:
l_status_dict = defaultdict(int)
for row in df_default.itertuples():
    # print(row[3])
    if row[18] not in l_status_dict:
        l_status_dict[row[18]] += 1
        
print(l_status_dict.keys())
print(len(ax))
# print(df.keys)

dict_keys(['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)', 'Does not meet the credit policy. Status:Charged Off'])
76


In [51]:
# a = list(zip(X_train[:50],y_train[:50]))
# print(a)
# pipline to scale the feature vectors and run the svm
rbf_svm_clf = Pipeline((
        ('imputer', Imputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ("rbf_svc", SVC(kernel='rbf', C=1))
    ))

linear_svm_clf = Pipeline((
        ('imputer', Imputer(strategy='mean')),
        ('scaler',StandardScaler()),
        ('linear_svc', LinearSVC(C=1))
    ))


In [36]:
# clf = svm.SVC(C=1, kernel='rbf')
# clf.fit(scaler.transform(X_train), y_train)
rbf_svm_clf.fit(X_train, y_train)
train_predictions = rbf_svm_clf.predict(X_train)

# test_predictions = clf.predict(X_test)
print(find_results(train_predictions,y_train))

0.64213


In [37]:
# rbf_svm_clf_C1000.fit(X_train, y_train)
# train_predictions_C1000 = rbf_svm_clf_C1000.predict(X_train)
linear_svm_clf.fit(X_train, y_train)
train_lin_predict = linear_svm_clf.predict(X_train)
print(find_results(train_lin_predict,y_train))

0.63753


In [52]:
# prep the SVM to run on the validation set
X_valid = []
for row in data_validation.itertuples():
    X_valid.append(feature(row))
    
y_valid = []
for row in data_validation.itertuples():
    y_valid.append(prediction_of_y(row))

    
#find the acc of the validation set
valid_predictions = rbf_svm_clf.predict(X_valid)

NotFittedError: This Imputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
precision, recall, fscore, support = score(y_train, train_predictions)
acc = accuracy_score(y_train, train_predictions)
print('precision is: '+str(precision))
print('recall is: '+str(recall))
print('fscore is: '+str(fscore))
print('support is: '+str(support))
print('accuracy is: '+str(acc))

In [ ]:
precision, recall, fscore, support = score(y_valid, valid_predictions)
acc = accuracy_score(y_valid, valid_predictions)
print('precision is: '+str(precision))
print('recall is: '+str(recall))
print('fscore is: '+str(fscore))
print('support is: '+str(support))
print('accuracy is: '+str(acc))

In [ ]:
# prep the SVM to run on the validation set
X_test = []
for row in df_test.itertuples():
    X_test.append(feature(row))
    
y_test = []
for row in df_test.itertuples():
    y_test.append(prediction_of_y(row))

    
#find the acc of the validation set
test_predictions = rbf_svm_clf.predict(X_test)

precision, recall, fscore, support = score(y_test, test_predictions)
acc = accuracy_score(y_test, test_predictions)
print('precision is: '+str(precision))
print('recall is: '+str(recall))
print('fscore is: '+str(fscore))
print('support is: '+str(support))
print('accuracy is: '+str(acc))

In [ ]:
print(sum(y_test))
print(len(y_test))